In [ ]:
using JuMP, Ipopt, Plots

#### Nonisothermal Batch Reactor
Based on: Example 8.4 of Biegler, L. T. (2010). Nonlinear programming: concepts, algorithms, and applications to chemical processes

Consider the nonisothermal batch reactor with first order parallel reactions $A \rightarrow B$, and $A \rightarrow C$. For optimal operation, we seek a temperature profile that maximizes the final amount of product $B$. The optimal control problem can be stated as: 

$\underset{T}{\max} \quad C_B(T_{end})$ 

subject to:

$ \dfrac{d C_A(t)}{d t} = -k_{1} \exp\bigg(\dfrac{-E_1}{RT}\bigg) C_A(t) - k_{2} \exp\bigg(\dfrac{-E_2}{RT}\bigg) C_A(t) \quad\quad t \in [0, T]$ <br>
$ \dfrac{d C_B(t)}{d t} = k_{1} \exp\bigg(\dfrac{-E_1}{RT}\bigg) C_A(t)  \quad\quad t \in [0, T]$ <br>
$ C_A(0) = C_{A,0}, \ C_B(0) = C_{B,0} $ <br>

where, $C_X$ denotes the concentration of species $X$, $T$ the reactor temperature (degree of freedom), $k_{1}$ and $k_{2}$ are the pre-exponential factor $E_1$ and $E_2$ are the activation energies of the reactions.$R$ is the universal gas constant. 

The problem above can be rewritten as:


$\underset{u(t)}{\min} \quad - C_B(T_{end})$ 

subject to:

$ \dfrac{d C_A(t)}{d t} = -C_A(t)(u(t) + k u(t)^\beta) \quad\quad t \in [0, T]$ <br>
$ \dfrac{d C_B(t)}{d t} = u(t) C_A(t)  \quad\quad t \in [0, T]$ <br>
$ C_A(0) = C_{A,0}, \ C_B(0) = C_{B,0} $ <br>


In [ ]:
#System parameters
nx     = 2          # [-] number of states
nfe    = 20          # number of control intervals
ncp    = 3           # number of collocation points
th     = 1          # time horizon
h      = th/nfe      # length of one finite element on the time horizon

ts     = Vector{Float64}(undef,nfe) # time series for plotting
for i in 1:nfe
    ts[i] = h*i
end

# Initial conditions
x0 = [1.0,0.0]
u0 = 2.0

# model parameters
k = 2.0
β = 2.0

In [ ]:
# Collocation parameters and radau time series
colmat = [0.19681547722366  -0.06553542585020 0.02377097434822;
          0.39442431473909  0.29207341166523 -0.04154875212600;
          0.37640306270047  0.51248582618842 0.11111111111111]
radau  = [0.15505 0.64495 1.00000]

In [ ]:
# JuMP model
m = Model(Ipopt.Optimizer)
set_optimizer_attribute(m, "warm_start_init_point", "yes")
set_optimizer_attribute(m, "print_level", 5)

# Set up variables
@variables(m, begin
    x[1:nx, 1:nfe, 1:ncp] ≥ 0 
    xdot[1:nx, 1:nfe, 1:ncp]
    0 <= u[1:nfe] <= 5
end)

# Set up initial guesses for solver
for i in 1:nfe
    for j in 1:ncp
        set_start_value(x[1,i,j], x0[1])
        set_start_value(x[2,i,j], x0[2])
    end
    set_start_value(u[i], u0)
end

# Set up objective function
@NLobjective(m, Min, -x[2,nfe,ncp])

#Set up the constraints
@NLconstraints(m, begin
    # set up differential equations
    m1[i=1:nfe, j=1:ncp], xdot[1,i,j] == -x[1,i,j]*(u[i] + k*u[i]^β)
    m2[i=1:nfe, j=1:ncp], xdot[2,i,j] == x[1,i,j]*u[i] 
    # set up collocation equations - 2nd-to-nth point
    coll_c_n[l=1:nx, i=2:nfe, j=1:ncp], x[l,i,j] == x[l,i-1,ncp] + h*sum(colmat[j,k]*xdot[l,i,k] for k in 1:ncp)
    # set up collocation equations - 1st point
    coll_c_0[l=1:nx, i=[1], j=1:ncp], x[l,i,j] == x0[l] + h*sum(colmat[j,k]*xdot[l,i,k] for k in 1:ncp)
    end);

In [ ]:
# Solve the model
solveNLP = JuMP.optimize!
status = solveNLP(m)

# Get values for plotting

res = Matrix{Float64}(undef,nx,nfe) # time series for plotting
for i in 1:nx
    res[i,:] = JuMP.value.(x[i,:,3])
end
uStar = JuMP.value.(u);

In [ ]:
gr()

p11 = plot(ts,res[1,:],linewidth=5,xaxis="time (t)",yaxis="x1(u,t)",legend=false)
p12 = plot(ts,res[2,:],linewidth=5,xaxis="time (t)",yaxis="x2(u,t)",legend=false)

g1 = plot(p11,p12,layout=(2,1))
display(g1)

g2 = plot(ts,uStar,linetype=:steppre,linewidth=5,xaxis="time [min]",yaxis="u",legend=false)
display(g2)